<a href="https://colab.research.google.com/github/dashang/16thNov-RAG-with-LLAMAIndex/blob/main/%5B16th_Nov_2025%5D_RAG_with_LLAMA_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install llama-index-core

In [2]:
pip install llama-index-llms-google-genai

  Using cached llama_index_llms_google_genai-0.7.3-py3-none-any.whl.metadata (3.0 kB)
Using cached llama_index_llms_google_genai-0.7.3-py3-none-any.whl (13 kB)


In [3]:
pip install llama-index-embeddings-google-genai

In [4]:
pip install google-generativeai

In [5]:
pip install sentence-transformers

In [6]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 72.2 MB/s eta 0:00:00


In [7]:
pip install llama-index-embeddings-openai

In [8]:
pip install llama-index-embeddings-huggingface

In [9]:
import os
GEMINI_API_KEY = "___"
os.environ["OPENAI_API_KEY"] = "__"

In [10]:
from llama_index.core import Document, Settings, VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from textwrap import shorten
# Use local embeddings (no API costs for this demo)
Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")
Settings.llm = None  # No LLM needed yet

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [11]:
# Demo 1 :
# Create documents with rich metadata
docs = [
    Document(
        text="Turmeric coffee (golden latte) combines coffee with turmeric and spices like cinnamon and black pepper. The result is earthy, slightly peppery, and anti-inflammatory.",
        metadata={"heading": "Turmeric Coffee", "category": "Specialty", "region": "India"}
    ),
    Document(
        text="Ashwagandha coffee blends adaptogenic herbs with coffee for sustained energy without jitters. Popular in Ayurvedic wellness circles.",
        metadata={"heading": "Ashwagandha Coffee", "category": "Adaptogen", "region": "India"}
    ),
    Document(
        text="Brewing temperature matters: lighter roasts need 92-96°C while darker roasts can handle 96-100°C. Hotter water extracts more but risks bitterness.",
        metadata={"heading": "Brewing Basics", "category": "Technique", "region": "Universal"}
    ),
]


In [12]:
index = VectorStoreIndex.from_documents(docs)

In [13]:
# Function to peek inside a Node
def inspect_node(index, query, label="Node"):
    """Show the internal structure of a Node"""
    retriever = index.as_retriever(similarity_top_k=1)
    results = retriever.retrieve(query)

    if results:
        node = results[0].node
        print(f"\n{'='*70}")
        print(f"🔍 {label} for query: '{query}'")
        print(f"{'='*70}")
        print(f"📦 Node ID: {node.node_id[:40]}...")
        print(f"📝 Text (truncated):\n   {shorten(node.get_content(), width=200, placeholder='...')}")
        print(f"🏷️  Metadata:")
        for key, val in node.metadata.items():
            print(f"   - {key}: {val}")
        print(f"📊 Relevance Score: {results[0].score:.4f}")



In [14]:
inspect_node(index,"Ashwagandha")


🔍 Node for query: 'Ashwagandha'
📦 Node ID: 7a39f269-1cc5-460c-ab9f-c3546d11f705...
📝 Text (truncated):
   Ashwagandha coffee blends adaptogenic herbs with coffee for sustained energy without jitters. Popular in Ayurvedic wellness circles.
🏷️  Metadata:
   - heading: Ashwagandha Coffee
   - category: Adaptogen
   - region: India
📊 Relevance Score: 0.6212


In [15]:
inspect_node(index,"Brewing Temperature")


🔍 Node for query: 'Brewing Temperature'
📦 Node ID: 7b47b340-b3d5-42ac-ab3f-90e58ac62fd7...
📝 Text (truncated):
   Brewing temperature matters: lighter roasts need 92-96°C while darker roasts can handle 96-100°C. Hotter water extracts more but risks bitterness.
🏷️  Metadata:
   - heading: Brewing Basics
   - category: Technique
   - region: Universal
📊 Relevance Score: 0.7023


In [ ]:
# Demo 2 : Four indexes

In [16]:
from llama_index.core import SimpleDirectoryReader, SummaryIndex, KeywordTableIndex, TreeIndex

In [18]:
docs = SimpleDirectoryReader(input_dir="./coffee_pages",required_exts = [".html"]).load_data()

In [19]:
len(docs)

15

In [20]:
# 1.
vector_idx = VectorStoreIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/30 [00:00<?, ?it/s]

In [21]:
# 2.
summary_idx = SummaryIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
# 3.
keyword_idx = KeywordTableIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Extracting keywords from nodes:   0%|          | 0/30 [00:00<?, ?it/s]

In [23]:
# 4.
tree_idx = TreeIndex.from_documents(docs,show_progress=True)

Parsing nodes:   0%|          | 0/15 [00:00<?, ?it/s]

Generating summaries:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 4 Index comparisons :

In [24]:
def detailed_comparison(query):
  # 1. Vector
  vec_results = vector_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Vector :",vec_results)
  # 2. Summary
  summary_results = summary_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Summary :",summary_results)
  # 3. Keyword
  keyword_results = keyword_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Keyword :",keyword_results)
  # 4. Tree
  tree_results = tree_idx.as_retriever(similarity_top_k=2).retrieve(query)
  print("Tree :",tree_results)

In [25]:
detailed_comparison("What is Ashwagandha coffee?")

Vector : [NodeWithScore(node=TextNode(id_='b24ff40a-34e3-4040-8f2e-984869c0201a', embedding=None, metadata={'file_path': '/content/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html', 'file_name': '01_ashwagandha_coffee_adaptogenic_latte.html', 'file_type': 'text/html', 'file_size': 4757, 'creation_date': '2025-11-16', 'last_modified_date': '2025-11-16'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='884c202d-6038-4eac-9c51-f4c2d1df3b21', node_type='4', metadata={'file_path': '/content/coffee_pages/01_ashwagandha_coffee_adaptogenic_latte.html', 'file_name': '01_ashwagandha_coffee_adaptogenic_latte.html', 'file_type': 'text/html', 'file_size': 4757, 'creation_date': '2025-11-16', 'last_modified_date

In [26]:
def detailed_comparison(query):
    """Deep comparison showing HOW each index type works"""
    print(f"\n{'='*70}")
    print(f"🔍 Query: '{query}'")
    print(f"{'='*70}")

    # 1. VectorStoreIndex (semantic)
    print("\n1️⃣  VectorStoreIndex (Semantic Search via FAISS)")
    print("   How it works: Query → embedding → find closest vectors in FAISS")
    vec_results = vector_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(vec_results, 1):
        print(f"   Match {i} (score: {r.score:.4f}):")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 2. SummaryIndex (overview)
    print("\n2️⃣  SummaryIndex (Pre-computed Summaries)")
    print("   How it works: Return stored summaries (no search needed)")
    sum_results = summary_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(sum_results, 1):
        print(f"   Summary {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 3. KeywordTableIndex (exact match)
    print("\n3️⃣  KeywordTableIndex (Exact Keyword Match)")
    print("   How it works: Extract keywords from query → lookup in table")
    kw_results = keyword_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(kw_results, 1):
        print(f"   Keyword match {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    # 4. TreeIndex (hierarchical)
    print("\n4️⃣  TreeIndex (Hierarchical Navigation)")
    print("   How it works: Traverse tree from root → relevant branch → leaf")
    tree_results = tree_idx.as_retriever(similarity_top_k=2).retrieve(query)
    for i, r in enumerate(tree_results, 1):
        print(f"   Branch {i}:")
        print(f"   {shorten(r.node.get_content(), width=150, placeholder='...')}")

    print(f"\n{'─'*70}")
    print("💡 Notice the differences!")

# Test with different query types
# detailed_comparison("What is Ashwagandha coffee?")
detailed_comparison("What temperature is ideal for light roasts?")


🔍 Query: 'What temperature is ideal for light roasts?'

1️⃣  VectorStoreIndex (Semantic Search via FAISS)
   How it works: Query → embedding → find closest vectors in FAISS
   Match 1 (score: 0.2514):
   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...
   Match 2 (score: 0.2311):
   It is not medical advice. Individuals with health conditions, pregnant or nursing, or taking medications should consult a qualified professional...

2️⃣  SummaryIndex (Pre-computed Summaries)
   How it works: Return stored summaries (no search needed)
   Summary 1:
   <!DOCTYPE html> <html lang="en"> <head> <meta charset="utf-8" /> <title>Ashwagandha Coffee (Adaptogenic Latte)</title> <meta name="description"...
   Summary 2:
   If using plant milks, choose barista blends for better foam and mouthfeel.</p> </section> <section> <h2>Method</h2> <ol> <li>Warm the milk gently;...
   Summary 3:
   <!DOCTYPE html> 

In [27]:
# Demo 4. Tree Indexes
# Create hierarchical documents (simulating chapters/sections)
hierarchical_docs = [
    Document(
        text="OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).",
        metadata={"level": "root", "title": "Coffee Guide Overview"}
    ),
    Document(
        text="CHAPTER 1 - COFFEE TYPES: Explores specialty coffees including turmeric lattes, ashwagandha blends, and saffron infusions. Each offers unique flavors and benefits.",
        metadata={"level": "chapter", "title": "Coffee Types", "chapter": 1}
    ),
    Document(
        text="Section 1.1 - Turmeric Coffee: Golden lattes combine turmeric with black pepper for bioavailability. Anti-inflammatory properties make it popular in wellness circles.",
        metadata={"level": "section", "chapter": 1, "section": 1}
    ),
    Document(
        text="Section 1.2 - Ashwagandha Coffee: Adaptogenic herbs reduce stress while coffee provides energy. Popular in Ayurvedic traditions for balanced stimulation.",
        metadata={"level": "section", "chapter": 1, "section": 2}
    ),
    Document(
        text="CHAPTER 2 - BREWING METHODS: Covers optimal temperatures (92-96°C for light roasts, 96-100°C for dark), grind sizes (fine for espresso, coarse for French press), and equipment choices.",
        metadata={"level": "chapter", "title": "Brewing Methods", "chapter": 2}
    ),
    Document(
        text="Section 2.1 - Temperature Control: Light roasts extract best at 92-96°C. Higher temps risk bitterness. Water quality matters as much as temperature.",
        metadata={"level": "section", "chapter": 2, "section": 1}
    ),
]


In [28]:
tree_idx_detailed = TreeIndex.from_documents(hierarchical_docs)

In [29]:
results = tree_idx_detailed.as_retriever(similarity_topk=4).retrieve("What temperature is ideal for light roasts?")

# Try this with all other indexes : Akash to action

In [30]:
results[0]

NodeWithScore(node=TextNode(id_='67c8d74f-f4f9-40e7-b717-e2e663764366', embedding=None, metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='81e08b69-b5d2-418e-aadd-11f352e5a629', node_type='4', metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, hash='e3fd92313c90d6899d8774bf3d8dedc510ccf22eb674ada1a3680024f635a590')}, metadata_template='{key}: {value}', metadata_separator='\n', text='OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).', mimetype='text/plain', start_char_idx=0, end_char_idx=156, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)

In [31]:
results[0].node.get_content()

'OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).'

In [32]:
results

[NodeWithScore(node=TextNode(id_='67c8d74f-f4f9-40e7-b717-e2e663764366', embedding=None, metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='81e08b69-b5d2-418e-aadd-11f352e5a629', node_type='4', metadata={'level': 'root', 'title': 'Coffee Guide Overview'}, hash='e3fd92313c90d6899d8774bf3d8dedc510ccf22eb674ada1a3680024f635a590')}, metadata_template='{key}: {value}', metadata_separator='\n', text='OVERVIEW: This guide covers coffee types (specialty, traditional), brewing methods (temperature, equipment), and health benefits (antioxidants, adaptogens).', mimetype='text/plain', start_char_idx=0, end_char_idx=156, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), score=None)]

In [ ]:
# wihtout tree -> search all the text
# with tree -> navigate smartly

In [33]:
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from llama_index.llms.google_genai import GoogleGenAI
import numpy as np

In [34]:
# Embeddings :
Settings.embed_model = GoogleGenAIEmbedding(
    model_name="models/text-embedding-004",
    api_key=GEMINI_API_KEY
)

In [35]:
# Embed sample texts
texts = [
    "Turmeric coffee with golden spices",
    "Golden latte with turmeric",  # Similar to above
    "Quantum entanglement in physics",  # Very different
]

print("\n📍 Embedding texts and comparing...")
embeddings = [Settings.embed_model.get_text_embedding(t) for t in texts]

# Calculate cosine similarity
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

sim_1_2 = cosine_similarity(embeddings[0], embeddings[1])
sim_1_3 = cosine_similarity(embeddings[0], embeddings[2])

print(f"\n📊 Similarity Scores:")
print(f"   '{texts[0][:40]}' ↔️")
print(f"   '{texts[1][:40]}'")
print(f"   Similarity: {sim_1_2:.4f} (HIGH - they mean the same!)")

print(f"\n   '{texts[0][:40]}' ↔️")
print(f"   '{texts[2][:40]}'")
print(f"   Similarity: {sim_1_3:.4f} (LOW - totally different!)")

print("\n💡 Key Takeaway: Embeddings capture MEANING, not just words!")

# Set up LLM for remaining demos
Settings.llm = GoogleGenAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0
)
print("\n✅ Gemini LLM configured (gemini-2.5-flash)")

# Rebuild indexes with Gemini embeddings
print("\n🔄 Rebuilding indexes with production Gemini embeddings...")
vector_idx = VectorStoreIndex.from_documents(docs)
print("✅ Indexes rebuilt with Gemini embeddings")




📍 Embedding texts and comparing...

📊 Similarity Scores:
   'Turmeric coffee with golden spices' ↔️
   'Golden latte with turmeric'
   Similarity: 0.9381 (HIGH - they mean the same!)

   'Turmeric coffee with golden spices' ↔️
   'Quantum entanglement in physics'
   Similarity: 0.5552 (LOW - totally different!)

💡 Key Takeaway: Embeddings capture MEANING, not just words!

✅ Gemini LLM configured (gemini-2.5-flash)

🔄 Rebuilding indexes with production Gemini embeddings...
✅ Indexes rebuilt with Gemini embeddings


In [36]:
# normal v/s semnatic chunking :
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser

In [37]:
# normal chunking
normal_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
normal_nodes = normal_splitter.get_nodes_from_documents(docs[:3])

# semantic chunking
semantic_splitter = SemanticSplitterNodeParser(embed_model = Settings.embed_model)
semnatic_nodes = semantic_splitter.get_nodes_from_documents(docs[:3])

In [38]:
from llama_index.core.retrievers import QueryFusionRetriever

In [39]:
import nest_asyncio
nest_asyncio.apply()

# Create base retriever
base_retriever = vector_idx.as_retriever(similarity_top_k=5)

# QueryFusionRetriever automatically generates query variations
fusion_retriever = QueryFusionRetriever(
    retrievers=[base_retriever],
    similarity_top_k=5,
    num_queries=3,  # Generate 3 variations
    mode="reciprocal_rerank",
    use_async=False,
    llm=Settings.llm
)

print("✅ Multi-query retriever created")
print("   - Generates 3 query variations automatically")
print("   - Retrieves with each variation")
print("   - Fuses results with reciprocal ranking")

# Test it
query = "What are the benefits of Ashwagandha?"
print(f"\n🔍 Original Query: '{query}'")

# Compare single vs multi-query
single_results = base_retriever.retrieve(query)
multi_results = fusion_retriever.retrieve(query)

print(f"\n📊 Results Comparison:")
print(f"   Single query: {len(single_results)} documents")
print(f"   Multi-query:  {len(multi_results)} unique documents")
print(f"   Improvement:  +{len(multi_results) - len(single_results)} docs")

✅ Multi-query retriever created
   - Generates 3 query variations automatically
   - Retrieves with each variation
   - Fuses results with reciprocal ranking

🔍 Original Query: 'What are the benefits of Ashwagandha?'

📊 Results Comparison:
   Single query: 5 documents
   Multi-query:  5 unique documents
   Improvement:  +0 docs


In [40]:
query = "What are the benefits of Ashwagandha?"
queries = fusion_retriever._get_queries(query)

print("\n🧩 Query Variations Generated:")
for i, q in enumerate(queries, 1):
    print(f"{i}. {q}")



🧩 Query Variations Generated:
1. Ashwagandha health benefits
2. Uses and effects of Ashwagandha


In [41]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


# -----------------------------
# 1. Create Query Engines
# -----------------------------
vector_qe = vector_idx.as_query_engine(similarity_top_k=4)
keyword_qe = keyword_idx.as_query_engine()
summary_qe = summary_idx.as_query_engine()


# -----------------------------
# 2. Wrap them as Tools
# -----------------------------
tools = [
    QueryEngineTool.from_defaults(
        query_engine=vector_qe,
        description="Use for general questions about coffee types and ingredients.",
        name="vector_engine"
    ),
    QueryEngineTool.from_defaults(
        query_engine=keyword_qe,
        description="Use for finding exact keywords or specific terms.",
        name="keyword_engine"
    ),
    QueryEngineTool.from_defaults(
        query_engine=summary_qe,
        description="Use for high-level overviews and summaries.",
        name="summary_engine"
    )
]


# -----------------------------
# 3. Create Router
# -----------------------------
selector = LLMSingleSelector.from_defaults(llm=Settings.llm)

router = RouterQueryEngine.from_defaults(
    query_engine_tools=tools,
    selector=selector,
    llm=Settings.llm,
    verbose = True
)

print("✅ Router created with 3 specialized engines")


# -----------------------------
# 4. Test Routing (Clean)
# -----------------------------
test_queries = [
    "What is turmeric coffee?", # vector search
    "Find entries with ashwagandha", # keyword search
    "Give me an overview of these documents" # summary search
]

print("\n🔀 Testing Query Routing...")

for q in test_queries:
    ans = router.query(q)
    print(f"\nQuery: {q}")
    print("Answer:", str(ans)[:200], "...")


✅ Router created with 3 specialized engines

🔀 Testing Query Routing...
Selecting query engine 0: The question 'What is turmeric coffee?' asks for a description or definition of a specific type of coffee. Choice (1) explicitly states 'Use for general questions about coffee types and ingredients,' which directly aligns with inquiring about 'turmeric coffee' as a 'coffee type' and implicitly about its 'ingredients' (turmeric). Choices (2) and (3) are less relevant as the question is not about finding exact keywords nor is it exclusively seeking a high-level overview without details about its nature or components..

Query: What is turmeric coffee?
Answer: Turmeric coffee combines the roasted notes of coffee with the warm, earthy characteristics of turmeric. It often includes a small amount of black pepper to enhance turmeric's flavor. ...
Selecting query engine 1: The question 'Find entries with ashwagandha' is specifically asking to locate entries containing a particular term or keyword 

In [ ]:
# 16 November

In [42]:
# =============================================================================
# DEMO 9: Post-Processing Pipeline
# =============================================================================
print("\n" + "="*70)
print("DEMO 9: Post-Processing Pipeline (Rerank + Dedup)")
print("="*70)

from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank

# Create post-processors
print("🔧 Setting up post-processors...")

reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=5
)
print("   ✅ Reranker (cross-encoder)")

deduplicator = SimilarityPostprocessor(similarity_cutoff=0.85)
print("   ✅ Deduplicator (similarity threshold 0.85)")

# Build query engine with post-processing
advanced_qe = vector_idx.as_query_engine(
    similarity_top_k=20,  # Retrieve more initially
    node_postprocessors=[
        reranker,      # Rerank to top 3
        deduplicator   # Remove duplicates
    ],
    response_mode="compact"
)


print("   1. Retrieve top 10 candidates")
print("   2. Rerank with cross-encoder to top 3")
print("   3. Deduplicate similar chunks")
print("   4. Synthesize final answer")

# Test it
query = "What are the health benefits of coffee with adaptogens?"
print(f"\n🔍 Query: '{query}'")
response = advanced_qe.query(query)
print(f"\n🤖 Answer:\n{response}")

print("\n💡 Key Takeaway: Post-processing = production-quality RAG!")

# TASK 1 : Try  the same question without reranker and deduplication!


DEMO 9: Post-Processing Pipeline (Rerank + Dedup)
🔧 Setting up post-processors...


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

   ✅ Reranker (cross-encoder)
   ✅ Deduplicator (similarity threshold 0.85)
   1. Retrieve top 10 candidates
   2. Rerank with cross-encoder to top 3
   3. Deduplicate similar chunks
   4. Synthesize final answer

🔍 Query: 'What are the health benefits of coffee with adaptogens?'

🤖 Answer:
The provided information describes the ingredients and preparation of a mushroom and ashwagandha coffee adaptogen blend, but it does not detail any specific health benefits associated with consuming it. The content explicitly states that it is not medical advice and advises consulting a qualified professional regarding health conditions or before trying new herbs.

💡 Key Takeaway: Post-processing = production-quality RAG!


In [45]:
# TASK 1 : Try  the same question without reranker and deduplication!


print("\n" + "="*70)
print("DEMO 9: Post-Processing Pipeline (without Rerank and Dedup)")
print("="*70)

from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank

# Create post-processors
print("🔧 Setting up post-processors...")

reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=5
)
print("   ✅ Reranker (cross-encoder)")

deduplicator = SimilarityPostprocessor(similarity_cutoff=0.85)
print("   ✅ Deduplicator (similarity threshold 0.85)")

# Build query engine with post-processing
advanced_qe1 = vector_idx.as_query_engine(
    similarity_top_k=20,  # Retrieve more initially
    node_postprocessors=[],
    response_mode="compact"
)


print("   1. Retrieve top 10 candidates")
print("   2. Rerank with cross-encoder to top 3")
print("   3. Deduplicate similar chunks")
print("   4. Synthesize final answer")

# Test it
query = "What are the health benefits of coffee with adaptogens?"
print(f"\n🔍 Query: '{query}'")
response1 = advanced_qe1.query(query)
print(f"\n🤖 Answer:\n{response1}")

print("\n💡 Key Takeaway: Post-processing = production-quality RAG!")



DEMO 9: Post-Processing Pipeline (without Rerank and Dedup)
🔧 Setting up post-processors...
   ✅ Reranker (cross-encoder)
   ✅ Deduplicator (similarity threshold 0.85)
   1. Retrieve top 10 candidates
   2. Rerank with cross-encoder to top 3
   3. Deduplicate similar chunks
   4. Synthesize final answer

🔍 Query: 'What are the health benefits of coffee with adaptogens?'

🤖 Answer:
The provided information does not describe the health benefits of coffee with adaptogens. It includes disclaimers stating that the content is not medical advice and advises individuals with health conditions, those who are pregnant or nursing, or those taking medications to consult a qualified professional before trying new herbs or routines. The "Benefits" sections for adaptogen blends focus on practical aspects, such as the ease of measuring tiny-dose powders or evaluating taste and routine adherence with small doses.

💡 Key Takeaway: Post-processing = production-quality RAG!


In [44]:
# 1 : 10 and 3
# The provided information describes the ingredients and
# preparation of a mushroom and ashwagandha coffee adaptogen blend,
# but it does not detail any specific health benefits associated with consuming it.
# The content explicitly states that it is not medical advice and advises consulting
# a qualified professional regarding health conditions or before trying new herbs.

# 2 : 20 and 5
# The provided information describes the blend as having a "modern adaptogenic profile"
# by layering earthy mushroom powders (such as lion’s mane or chaga) with ashwagandha.
# However, it does not detail specific health benefits associated with consuming coffee with adaptogens.


In [ ]:
# =============================================================================
# DEMO 10: Custom Grounded Prompts
# =============================================================================
print("\n" + "="*70)
print("DEMO 10: Custom Grounded Prompts")
print("="*70)

from llama_index.core import PromptTemplate

# Custom prompt template with strict grounding
qa_prompt_tmpl = """
You are a coffee expert assistant. Your goal is to provide accurate,
grounded answers based ONLY on the provided context.

STRICT RULES:
1. Use ONLY the context below - no outside knowledge
2. If the answer is not in the context, say exactly: "I don't have that information in my knowledge base."
3. Keep answers to 3-5 sentences
4. Add [Source 1], [Source 2] citations after each fact
5. Be conversational but precise

Context:
{context_str}

Question: {query_str}

Answer with citations:
"""

qa_prompt = PromptTemplate(qa_prompt_tmpl)

# Apply custom prompt to query engine
custom_qe = vector_idx.as_query_engine(
    text_qa_template=qa_prompt,
    similarity_top_k=4,
    response_mode="compact"
)

print("✅ Custom grounded prompt applied")

# Test with knowledge
print("\n📝 Test 1: Question with knowledge in corpus")
query1 = "What is Ashwagandha coffee?"
response1 = custom_qe.query(query1)
print(f"Query: {query1}")
print(f"Answer: {response1}")


DEMO 10: Custom Grounded Prompts
✅ Custom grounded prompt applied

📝 Test 1: Question with knowledge in corpus
Query: What is Ashwagandha coffee?
Answer: Ashwagandha coffee is a modern fusion trend that combines roasted coffee with powdered ashwagandha, an herb traditionally used in Indian traditions [Source 1]. The aim is to enjoy a familiar coffee experience while incorporating the herb's earthy and slightly bitter notes [Source 1]. This beverage is often prepared like a latte, with spices added to help balance the flavor [Source 1]. It reflects a broader movement of adapting herbs into everyday drinks [Source 1].


In [ ]:
# Test without knowledge
print("\n📝 Test 2: Question without knowledge in corpus")
query2 = "What is quantum entanglement in coffee brewing?"
response2 = custom_qe.query(query2)
print(f"Query: {query2}")
print(f"Answer: {response2}")

print("\n💡 Key Takeaway: Custom prompts prevent hallucinations with fallback strings!")


📝 Test 2: Question without knowledge in corpus
Query: What is quantum entanglement in coffee brewing?
Answer: I don't have that information in my knowledge base.

💡 Key Takeaway: Custom prompts prevent hallucinations with fallback strings!


In [ ]:
# DO NOT RUN!

In [46]:
# Create chat engine with built-in memory
memory = ChatMemoryBuffer.from_defaults(token_limit=3000)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="You are a helpful coffee expert. Keep answers to 3-5 sentences."
)

print("\n🚀 Running Conversation with Built-in Memory:\n")
print("-"*80)

# Turn 1
question1 = "What is Ashwagandha coffee?"
print(f"👤 USER: {question1}")
response1 = chat_engine.chat(question1)
print(f"🤖 BOT: {response1.response}\n")

NameError: name 'ChatMemoryBuffer' is not defined

In [ ]:
# Turn 2 - Uses "its" (requires memory)
question2 = "What are its health benefits?"
print(f"👤 USER: {question2}")
print("   💡 Note: 'its' refers to Ashwagandha coffee from previous turn")
response2 = chat_engine.chat(question2)
print(f"🤖 BOT: {response2.response}\n")


👤 USER: What are its health benefits?
   💡 Note: 'its' refers to Ashwagandha coffee from previous turn
🤖 BOT: Ashwagandha coffee is primarily known for its ability to provide sustained energy throughout the day. A key health benefit is that it helps to mitigate the jitters and anxious feelings often associated with regular coffee consumption. This is attributed to the adaptogenic properties of ashwagandha, which work to balance the body's response to stress. Its popularity in Ayurvedic wellness circles also suggests a general alignment with holistic well-being practices.



In [ ]:
question3 = "How should I brew it?"
print(f"👤 USER: {question3}")
print("   💡 Note: 'it' refers to Ashwagandha coffee")
response3 = chat_engine.chat(question3)
print(f"🤖 BOT: {response3.response}\n")


👤 USER: How should I brew it?
   💡 Note: 'it' refers to Ashwagandha coffee
🤖 BOT: The provided information describes what Ashwagandha coffee is and its benefits, but it does not offer specific instructions on how to brew it. The "Brewing Basics" section gives general temperature guidelines for different coffee roasts, suggesting 92-96°C for lighter roasts and 96-100°C for darker roasts. However, it doesn't detail methods or specific recommendations for Ashwagandha coffee blends. Therefore, I cannot provide specific brewing instructions based on the given context.



## TASK FOR US

In [47]:
from llama_index.core.memory import ChatMemoryBuffer

In [55]:
docs1 = SimpleDirectoryReader(input_dir="./harry",required_exts = [".pdf"]).load_data()

In [56]:
index1 = VectorStoreIndex.from_documents(docs1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [50]:
print(len(docs1))

1


In [66]:
# Create chat engine with built-in memory
memory = ChatMemoryBuffer.from_defaults(token_limit=3000)

chat_engine = index1.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt="You are a helpful harry potter spoiler expert. Keep answers to 3-5 sentences "
)

print("\n🚀 Running Conversation with Built-in Memory:\n")
print("-"*80)

# Turn 1
question1 = "Who is harry potter?"
print(f"👤 USER: {question1}")
response1 = chat_engine.chat(question1)
print(f"🤖 BOT: {response1.response}\n")


🚀 Running Conversation with Built-in Memory:

--------------------------------------------------------------------------------
👤 USER: Who is harry potter?
🤖 BOT: Harry Potter is the titular protagonist of J.K. Rowling's immensely popular fantasy book series. He is a young wizard who discovers his magical heritage on his eleventh birthday, learning he is famous in the wizarding world as "the boy who lived." Harry attends Hogwarts School of Witchcraft and Wizardry, where he befriends Ron Weasley and Hermione Granger. His destiny is intertwined with the dark wizard Lord Voldemort, who murdered his parents and attempted to kill him as a baby. Throughout the series, Harry confronts Voldemort multiple times, ultimately fulfilling a prophecy to defeat him.



In [61]:
# Turn 2 - Uses "its" (requires memory)
question2 = "What happens to its friends?"
print(f"👤 USER: {question2}")
print("   💡 Note: 'its' refers to harry from previous turn")
response2 = chat_engine.chat(question2)
print(f"🤖 BOT: {response2.response}\n")


👤 USER: What happens to its friends?
   💡 Note: 'its' refers to harry from previous turn
🤖 BOT: Out of Context. The provided text consists of unreadable data from a PDF file and does not contain any narrative information about Harry Potter or his friends. Therefore, I cannot tell you what happens to them based on this context.



In [62]:
question3 = "Where is shaktimaan?"
print(f"👤 USER: {question3}")
print("   💡 Note: it is out of context")
response3 = chat_engine.chat(question3)
print(f"🤖 BOT: {response3.response}\n")


👤 USER: Where is shaktimaan?
   💡 Note: it is out of context
🤖 BOT: Out of Context. The provided text is unreadable and does not contain any information about "Shaktimaan."



## Changing Chat behaviour